<a href="https://colab.research.google.com/github/JanMarcelKezmann/OpenAI-API-Guide/blob/main/Step_by_Step_Guide_ChatGPT_Python_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step-by-Step Guide ChatGPT Python API

Hi, this notebook should offer you a simple ready-to-use hands-on guide for utilizing the ChatGPT API in a Jupyter Notebook.

I have explained all the necessary steps in my Blog Post <a href="">Your 5-Minute Step-By-Step ChatGPT API Guide</a>.

This notebooks contains all the code mentioned in the blog article with additional comments from my side.

I hope you find it valueable.


# Imports

First install the *openai* library in this notebook.

In [1]:
!pip install --upgrade openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


After you have installed it just import it as shown below.

In [2]:
import openai

After the relevant packages are imported we can load and set the OpenAI API key from a text file.

I have stored it in a file called *openai-api-key.txt* in the same folder as the Jupyter Notebook itself. (If you are working in Google Colab as I did, then you have to mount your drive first to access the files in your Google Drive.)

Change the *file_path* variable if you have named your text file differently.

In [3]:
# Load your OpenAI API key from the text file
file_path = "drive/MyDrive/Blogs/ChatGPT API/openai-api-key.txt"

with open(file_path, "r") as f:
  openai.api_key = f.read().strip("\n")

# Embedding The OpenAI ChatGPT API In A Jupyter Notebook

Having set up the mandatory openai library and API key, we can now continue by making a test request to check whether everything is working properly.

Before being able to retrieve a response, we have to define the model first. In our case we would like to utilize the **gpt-3.5-turbo** model.

If you want to get a list of all models than just execute the code cell below

In [4]:
for model in openai.Model.list()["data"]:
    print(model.get("id", "---"))

babbage
davinci
text-davinci-edit-001
babbage-code-search-code
text-similarity-babbage-001
code-davinci-edit-001
text-davinci-001
ada
babbage-code-search-text
babbage-similarity
code-search-babbage-text-001
text-curie-001
whisper-1
code-search-babbage-code-001
text-davinci-003
text-ada-001
text-embedding-ada-002
text-similarity-ada-001
curie-instruct-beta
ada-code-search-code
ada-similarity
code-search-ada-text-001
text-search-ada-query-001
davinci-search-document
ada-code-search-text
text-search-ada-doc-001
davinci-instruct-beta
text-similarity-curie-001
code-search-ada-code-001
ada-search-query
text-search-davinci-query-001
curie-search-query
davinci-search-query
babbage-search-document
ada-search-document
text-search-curie-query-001
text-search-babbage-doc-001
curie-search-document
text-search-curie-doc-001
babbage-search-query
text-babbage-001
text-search-davinci-doc-001
text-search-babbage-query-001
curie-similarity
gpt-3.5-turbo-0301
curie
gpt-3.5-turbo
text-similarity-davinci-00

**Note:** Accessing the gpt-3.5-turbo model costs $0.002 per 1,000 tokens.

In [5]:
model = "gpt-3.5-turbo"

## Define A message

Crafting a message requires to steps:

1. Define the **role**
2. Define the **content**

The **role** can be one of three things: *system*, *user*, *assistant*

A short explanation:

 - *system*: Primes the chatbot to act in a specific way, e.g., 'act as a helpful mathematics professor'
 - *user*: This is basically *you*, and is used for all queries you make
 - *assistant*: This is a previous response of the chatbot, useful for conversations in which you might refer to the previous outputs

For our first example, we just want to query a response by asksing in the role of the *user*.

In [6]:
my_first_message = "How many people live in Brasil?"

message_list = [
    {
        "role": "user",
        "content": my_first_message
    }
]

## Get The Response

In [7]:
completion = openai.ChatCompletion.create(
    model=model,
    messages=message_list
)

Since we received no error message the query should have been successful. The complete response looks as follows:

In [8]:
completion

<OpenAIObject chat.completion id=chatcmpl-73i0L56JFDWj6E3FIHQEphCvGgmOE at 0x7faee8fa1400> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "As an AI language model, I do not have access to real-time data, but according to the United Nations (as of 2021), the estimated population of Brazil is 213.4 million.",
        "role": "assistant"
      }
    }
  ],
  "created": 1681118493,
  "id": "chatcmpl-73i0L56JFDWj6E3FIHQEphCvGgmOE",
  "model": "gpt-3.5-turbo-0301",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 41,
    "prompt_tokens": 15,
    "total_tokens": 56
  }
}

However, we are mostly interested into the answer we asked for, which you simply retrieve by executing the following line of code:

In [9]:
response = completion["choices"][0]["message"]["content"]
print(response)

As an AI language model, I do not have access to real-time data, but according to the United Nations (as of 2021), the estimated population of Brazil is 213.4 million.


**Note:** As you may have noticed we receive some additional valuable information in the response as well:

 - **role**: assistant (aka. the chatbot)
 - **model**: gpt-3.5-turbo-0301 (the specific model model name/id)
 - **usage**: a dictionary (relevant for calcualting the price of your queries):
    - *prompt_tokens*: the amount of tokens used in your query
    - *completion_tokens*: the amount of tokens used in the response
    - *total_token*: the total number of tokens

And that is it for the start.

Below I will provide you with a more sophisticated code that allows you to have a real chat-like conversation in your Jupyter Notebook or an application of your choice.

# Setting up a Chat

Setting up a chat actually won't require much extra work.

What we basically need to add is the option to provide the whole chat history as a message in order for the model to understand and know the whole context.

To do this I will use the *chat_history* variable - define below - to keep track of all the queries I made and responses I have received. Each time I ask something new or get a response I will append that content with the proper **role** to the list as a dictionary.

In [10]:
chat_history = []

In [11]:
def chat(user_input: str, chat_history: list = [], role: str = "user", model: str = "gpt-3.5-turbo") -> list:
    # add your new query to the chat history
    chat_history.append(
        {
            "role": role,
            "content": user_input
        }
    )

    # post a request to the openai API
    completion = openai.ChatCompletion.create(
        model=model,
        messages=chat_history
    )

    # get the response
    response = completion["choices"][0]["message"]["content"]   

    # append the response with the role - assistant - to the chat history
    chat_history.append(
        {
            "role": completion["choices"][0]["message"]["role"],
            "content": response
        }
    )

    # return the response and chat_history
    return response, chat_history

## Start chatting

### Prime the model with a system message

Having defined the chat function, I would test it with an example making asking two consecutive questions where the second relies on the context of the first and its response.

In addition, we will prime the model by using the *system* role, to hopefully get a better response.

In [12]:
# setting the role to system to prime the model
response, chat_history = chat(
    user_input = "Act as a helpful math teacher.",
    chat_history = chat_history,
    role = "system"
)

Let's print the response and chat history to see if everything worked as expected.

In [13]:
print(response)

Of course, I am here to help with any math questions you may have. What do you need help with?


In [14]:
chat_history

[{'role': 'system', 'content': 'Act as a helpful math teacher.'},
 {'role': 'assistant',
  'content': 'Of course, I am here to help with any math questions you may have. What do you need help with?'}]

### My Initial Question

The response as well as the chat_history looks as we wanted it, let's see if the AI model behaves as it was advised to.

In [15]:
response, chat_history = chat(
    user_input = "What is the derivative of the function f(x) = x * e^x",
    chat_history = chat_history
)

In [16]:
print(response)

To find the derivative of the function f(x) = x * e^x, we can use the product rule for differentiation, which states that the derivative of a product of two functions is equal to the first function times the derivative of the second function plus the second function times the derivative of the first function.

In this case, let u = x and v = e^x. Then, using the product rule, we get:

f'(x) = (u)' * v + u * (v)'

Where (u)' and (v)' are the derivatives of u and v respectively.

Taking the derivatives, we get:

f'(x) = 1 * e^x + x * e^x = (x + 1) * e^x

Therefore, the derivative of the function f(x) = x * e^x is (x + 1) * e^x.


Well, so far this not only looks factually and mathematically correct but the model also did what it should, meaning acting as a helpful math teacher.

By explaining each step which lead to the correct answer it did that perfectly.

### Follow-Up Question

Let's see if it can answer my follow up question which will require the understanding of the context from the previous question and respone.

In [17]:
response, chat_history = chat(
    user_input = "Can you now reverse your last calculation",
    chat_history = chat_history
)

In [18]:
print(response)

Sure, I can do that. 

To reverse the previous calculation, we need to find the antiderivative (or integral) of (x + 1) * e^x with respect to x. 

The antiderivative is found by using integration by parts, which is the opposite of the product rule for differentiation. 

Let u = x + 1 and dv/dx = e^x, so that du/dx = 1 and v = e^x. 

Then, using integration by parts, we have: 

∫ (x + 1) * e^x dx = u * v - ∫ v * du/dx dx 

= (x + 1) * e^x - ∫ 1 * e^x dx 

= (x + 1) * e^x - e^x + C 

where C is the constant of integration. 

Therefore, the antiderivative of (x + 1) * e^x with respect to x is (x + 1) * e^x - e^x + C.


First of all, the AI model understood the context and figured out that I wanted to integrate the function that has been previously derived.

Also, the model did everything correct even though it could have summarized the final sum (x + 1) * e^x - e^x + C to x * e^x + C.

I could now ask to correct itself, but I will stop for now as I have just shown how to set up the chat functionality which was the main purpose of this guide.

Finally, let's look at the compelte chat history:

In [19]:
for el in chat_history:
    str_content = el["content"].replace("\n", "\n            | ")
    print(f'{el["role"].capitalize()}:{" " * (11 - len(el["role"]))}| {str_content}\n')

System:     | Act as a helpful math teacher.

Assistant:  | Of course, I am here to help with any math questions you may have. What do you need help with?

User:       | What is the derivative of the function f(x) = x * e^x

Assistant:  | To find the derivative of the function f(x) = x * e^x, we can use the product rule for differentiation, which states that the derivative of a product of two functions is equal to the first function times the derivative of the second function plus the second function times the derivative of the first function.
            | 
            | In this case, let u = x and v = e^x. Then, using the product rule, we get:
            | 
            | f'(x) = (u)' * v + u * (v)'
            | 
            | Where (u)' and (v)' are the derivatives of u and v respectively.
            | 
            | Taking the derivatives, we get:
            | 
            | f'(x) = 1 * e^x + x * e^x = (x + 1) * e^x
            | 
            | Therefore, the derivative of the f